______
## Species Preliminary Ingest

In [14]:
import sqlite3

# Database standard connection
conn=sqlite3.connect('nursery.db')
cursor=conn.cursor()

# Premilinar species data
species_data =     [
    ('Tabebuia rosea', 'Roble Rosado', 'Bignoniaceae', '2025-05-18', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Tabebuia impetiginosa', 'Roble Morado', 'Bignoniaceae', '2025-05-02', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Laburnum anagyroides', 'Lluvia de Oro', 'Fabaceae', '2025-06-22', 'De 8 a 10 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Mangifera indica', 'Mango', 'Anacardiaceae', '2025-07-06', 'Más de 15 días', 'Moderado', 'Medio', 'Pleno sol', 'Semilla'),
    ('Delonix regia', 'Acacia', 'Fabaceae', '2025-05-18', 'De 8 a 10 días', 'Rápido', 'Medio', 'Sombra parcial', 'Semilla'),
    ('Citrus × paradisi', 'Toronja', 'Rutaceae', '2025-05-11', 'Más de 15 días', 'Lento', 'Medio', 'Pleno sol', 'Semilla'),
    ('Carica papaya', 'Papaya', 'Caricaceae', '2025-05-11', 'Menos 8 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dypsis lutescens', 'Palma Areca', 'Arecaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Sombra parcial', 'Semilla'),
    ('Vachellia farnesiana', 'Aromo', 'Fabaceae', '2025-08-29', 'De 10 a 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena trifasciata', 'Lengua de Suegra', 'Asparagaceae', '2025-08-11', 'Más de 15 días', 'Lento', 'Bajo', 'Pleno sol', 'División'),
    ('Terminalia catappa', 'Almendro', 'Combretaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Zamioculcas zamiifolia', 'Millonaria', 'Araceae', '2025-08-29', 'De 10 a 15 días', 'Lento', 'Bajo', 'Sombra parcial', 'Esqueje'),
    ('Luffa aegyptiaca', 'Estropajo', 'Cucurbitaceae', '2025-08-11', 'De 8 a 10 días', 'Rápido', 'Alto', 'Pleno sol', 'Semilla'),
    ('Cassia grandis', 'Cañandonga', 'Fabaceae', '2025-08-29', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Semilla'),
    ('Dracaena marginata', 'Marginata Tricolor', 'Asparagaceae', '2025-09-10', 'Más de 15 días', 'Rápido', 'Medio', 'Pleno sol', 'Esqueje'),
    ('Tradescantia pallida', 'Purpurina', 'Commelinaceae', '2025-09-10', 'Menos 8 días', 'Moderado', 'Bajo', 'Pleno sol', 'Esqueje')
]

# parametrized query for multiple row insertion
insert_query = '''
    INSERT INTO ESPECIE_LOTE
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO, LUZ, PROPAGACION)
    VALUES
        (?,?,?,?,?,?,?,?,?)
'''

# Excecute in transaction
with sqlite3.connect('nursery.db') as conn:
    conn.executemany(insert_query, species_data)

In [5]:
import sqlite3

def insert_specie(datos_especie):
    """
    Insert a new species to the ESPECIE_LOTE table

    Arg:
        datos_especie: Tuple with the data to import acording to the table
    """
    try:
        # Connect to the database
        conn = sqlite3.connect('nursery.db')
        cursor = conn.cursor()

        insert_query = ''' 
        INSERT INTO ESPECIE_LOTE 
        (NOMBRE_CIENTIFICO, NOMBRE_COMUN, FAMILIA, FECHA_INICIO, 
        GERMINACION_PROMEDIO, CRECIMIENTO, RIEGO,LUZ, NOTAS)
        VALUES (?,?,?,?,?,?,?,?,?)
        '''
        # Execute the insertion
        cursor.execute(insert_query, datos_especie)
        conn.commit()

        print("Especie insertada correctamente!")

    except sqlite3.Error as error:
        print(f"Error al insertar: {error}")
    
    finally:
        # close connection
        if conn:
            conn.close()

# New species data
new_data = (
    "Tradescantia pallida", # NOMBRE_CIENTIFICO
    "Purpurina",  # NOMBRE_COMÚN
    "Commelinaceae",     # FAMILIA
    "2025-09-10",      # FECHA_INICIO
    "Menos 8 días",  # GERMINACIÓN_PROMEDIO
    "Moderado",          # CRECIMIENTO
    "Bajo",           # RIEGO
    "Pleno sol",       # LUZ
    "Ingreso de 10 plantas" # NOTAS
)
    
insert_specie(new_data)

Especie insertada correctamente!


_______
## Batch Preliminary Ingest

In [ ]:
import sqlite3

# Database standard connection
conn=sqlite3.connect('nursery.db')
cursor=conn.cursor()

# Preliminar batch data
batch_data=[
    ('2', '', '06-22-2025', '200', '06-28-2025', '07-19-2025', '91'),
    ('3', '', '06-06-2025', '101', '06-29-2025', '07-24-2025', '38'),
    ('1', '', '05-02-2025', '300', '05-20-2025', '06-08-2025', '101'),
    ('1', '', '05-15-2025','80','05-29-2025', '06-19-2025', '57'),
    ('4', '','05-22-2025','15','06-01-2025','06-19-2025', '7'),
    ('5', '', '04-30-2025', '70', '05-18-2025', '06-08-2025', '57')
]

# Parametrized query for multiple row insertion
insert_query = '''
INSERT INTO SPECIES
    (SPECIES_ID, BATCH_CODE, START_DATE, INITIAL_SEED_COUNT, SPROUT_DATE, BAGGING_DATE, INITIAL_PLANT_COUNT)
VALUES
    (?,?,?,?,?,?,?)
'''

# Excecute in transaction
with sqlite3.connect('nursery.db') as conn:
    conn.executemany(insert_query, batch_data)

_______
# Irrigation Ingest

In [ ]:
import sqlite3
from datetime import date

def log_daily_irrigation(irrigation_list, log_date=None):
    """
    A reusable function to log daily irrigation for MULTIPLE lots at once.
    
    Parameters:
    irrigation_list (list of tuples): Each tuple is (lote_id, water_lt, notes).
    log_date (str, optional): Date in 'DD-MM-YYYY' format. Uses today if not provided.
    """
    # Use today's date if no specific date is provided
    if log_date is None:
        log_date = date.today().strftime("%d-%m-%Y")
    
    conn = sqlite3.connect('nursery.db')
    cursor = conn.cursor()
    
    success_count = 0
    total_water = 0.0
    
    try:
        # Calculate total water first
        total_water = sum(water_lt for (lote_id, water_lt, notes) in irrigation_list)
        
        # Prepare the query
        insert_query = '''
            INSERT INTO REGISTRO_RIEGO (ESPECIE_LOTE_ID, FECHA, CANTIDAD_AGUA_LT, NOTAS)
            VALUES (?, ?, ?, ?)
        '''
        
        # Prepare the data: add the same date to every entry
        data_to_insert = [(lote_id, log_date, water_lt, notes) for (lote_id, water_lt, notes) in irrigation_list]
        
        # Execute all inserts at once
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        
        success_count = len(irrigation_list)
        print(f"✅ Successfully logged irrigation for {success_count} lots on {log_date}.")
        print(f"💧 Total water used: {total_water} liters")
        
    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()
    
    return success_count, total_water

# --- HOW TO USE IT DAILY ---
# 1. Define your irrigation for the day in this list
# Format: (lote_id, water_amount, notes)
today_irrigation = [
    (1, 5, None),
    #(2, 5,None),
    (4, 0.5, None),
    #(5, 0.5, None),
    (6, 2, None),
    (7, 0.5, None),
    (8, 1.5, None),
    #(9, 3, None),
    #(10, 2, None),
    (11, 1.5, None),
    (13, 0.5 , None),
    #(14, 0.5, None),
    #(15, 0.5, None),
    #(16, 1, None),
    # Add more lots as needed...
]

# 2. Run this single function call
log_daily_irrigation(today_irrigation)

# --- FOR BACKFILLING PAST DATES ---
# If you need to log for a different date, just provide the date:
# last_weeks_irrigation = [(3, 4.0, ''), (4, 1.5, '')]
# log_daily_irrigation(last_weeks_irrigation, log_date='25-08-2025')

✅ Successfully logged irrigation for 8 lots on 16-09-2025.
💧 Total water used: 17.5 liters


(8, 17.5)

In [4]:
import sqlite3

conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# Insert query to nullify specific mistakes
cursor.execute('''
    DELETE FROM ESPECIE_LOTE 
    WHERE NOMBRE_CIENTIFICO = 'Tradescantia pallida'
''')

conn.commit()
conn.close()
